In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install git+https://github.com/PyFstat/PyFstat@python37

In [ ]:
!pwd

In [ ]:
!mkdir test-github-clone

In [ ]:
!ls

In [ ]:
!cd test-github-clone

In [ ]:
!pwd

In [ ]:
cd test-github-clone

In [ ]:
pwd

In [ ]:
!git clone https://github.com/PyFstat/PyFstat.git

In [ ]:
pwd


In [ ]:
cd ..

In [ ]:
pwd

In [ ]:
cp test-github-clone/examples/tutorials/tutorial_utils.py .

In [ ]:
pwd

In [ ]:
ls

In [ ]:
cp test-github-clone/PyFstat/examples/tutorials/tutorial_utils.py .

In [ ]:
ls

In [ ]:
import numpy as np

import pyfstat
from pyfstat.utils import get_sft_as_arrays

# Local module to simplify plotting
import tutorial_utils

logger = pyfstat.set_up_logger(label="0_generating_noise", log_level="INFO")

%matplotlib inline

#noise generation

# 1 Gausian Noise

In [ ]:
writer_kwargs = {
    "label": "single_detector_gaussian_noise",
    "outdir": "PyFstat_example_data",
    "tstart": 1238166018,  # Starting time of the observation [GPS time]
    "duration": 365 * 86400,  # Duration [seconds]
    "detectors": "H1,L1,V1",  # Detector to simulate, in this case LIGO Hanford
    "F0": 317.0,  # Central frequency of the band to be generated [Hz]
    "Band": 1.0,  # Frequency band-width around F0 [Hz]
    "sqrtSX": "1e-23,1e-24,1e-25",  # Single-sided Amplitude Spectral Density of the noise
    "Tsft": 1800,  # Fourier transform time duration
    "SFTWindowType": "tukey",  # Window function to compute short Fourier transforms
    "SFTWindowBeta": 0.01,  # Parameter associated to the window function
}
writer = pyfstat.Writer(**writer_kwargs)

writer.make_data()

frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

In [ ]:

# Read SFT data into numpy arrays and plot real and imaginary parts
frequency_noise, timestamps_noise, fourier_data_noise = get_sft_as_arrays(writer.sftfilepath)

# 2 Non-stationary noise

In [ ]:
'''segment_lengths = [5 * 86400, 3 * 86400, 4 * 86400]
segment_sqrtSX = [4e-23, 1e-23, 3e-23]

sft_path = []

# Setup Writer
writer_kwargs = {
    "outdir": "PyFstat_example_data",
    "tstart": 1238166018,
    "detectors": "H1,L1,V1",  # Detector to simulate, in this case LIGO Hanford
    "F0": 100.0,  # Central frequency of the band to be generated [Hz]
    "Band": 1.0,  # Frequency band-width around F0 [Hz]
    "sqrtSX": "1e-23,1e-24,1e-25",  # Single-sided Amplitude Spectral Density of the noise
    "Tsft": 1800,  # Fourier transform time duration
    "SFTWindowType": "tukey",
    "SFTWindowBeta": 0.01,
}

for segment in range(len(segment_lengths)):
    writer_kwargs["label"] = f"segment_{segment}"
    writer_kwargs["duration"] = segment_lengths[segment]
    writer_kwargs["sqrtSX"] = segment_sqrtSX[segment]

    if segment > 0:
        writer_kwargs["tstart"] += writer_kwargs["Tsft"] + segment_lengths[segment - 1]

    writer = pyfstat.Writer(**writer_kwargs)
    writer.make_data()

    sft_path.append(writer.sftfilepath)

sft_path = ";".join(sft_path)  # Concatenate different files using ;'''

In [ ]:
#frequency_noise, timestamps_noise, fourier_data_noise = get_sft_as_arrays(sft_path)



# 3 Narrow instrumental artifacts

In [ ]:
'''writer_kwargs = {
    "label": "single_detector_spectral_line",
    "outdir": "PyFstat_example_data",
    "tstart": 1238166018,  # Starting time of the observation [GPS time]
    "duration": 5 * 86400,  # Duration [seconds]
    "detectors": "V1",  # Detector to simulate, in this case LIGO Hanford
    "F0": 100.0,  # Central frequency of the band to be generated [Hz]
    "phi": 1.3,  # Initial phase of the spectral line
    "Band": 1.0,  # Frequency band-width around F0 [Hz]                "h0": 1e-24,              # Amplitude of the spectral line
    "sqrtSX": 1e-25,  # Single-sided Amplitude Spectral Density of the noise
    "Tsft": 1800,  # Fourier transform time duration
    "SFTWindowType": "tukey",
    "SFTWindowBeta": 0.01,
}

writer = pyfstat.LineWriter(**writer_kwargs)
writer.make_data()'''

In [ ]:
#frequency_noise, timestamps_noise, fourier_data_noise = get_sft_as_arrays(writer.sftfilepath)

# Generationg Signal Plus noise

In [ ]:
writer_kwargs = {
    "label": "single_detector_gaussian_noise",
    "outdir": "PyFstat_example_data",
    "tstart": 1238166018,
    "duration": 365 * 86400,
    "detectors": "H1,L1,V1",
    "Band": 1.0,  # Frequency band-width around F0 [Hz]
    "sqrtSX": "1e-23,1e-24,1e-25",
    "Tsft": 1800,
    "SFTWindowType": "tukey",
    "SFTWindowBeta": 0.01,
}

signal_parameters = {
    "F0": 317.0,
    "F1": -1e-9,
    "Alpha": 0.0,
    "Delta": 0.0,
    "h0": 1e-22,
    "cosi": 1,
    "psi": 0.0,
    "phi": 0.0,
    "tref": writer_kwargs["tstart"],
}

writer = pyfstat.Writer(**writer_kwargs, **signal_parameters)
writer.make_data()

In [ ]:
#Signal plus noise
frequency_signal, timestamps_signal, fourier_data_signal = get_sft_as_arrays(writer.sftfilepath)

In [ ]:
frequency_signal

In [ ]:
len(timestamps_signal['H1'])

In [ ]:
fourier_data_noise['V1']

Fourier compare with noise, signal+noise

In [ ]:
H1_signal_matrix=np.absolute(fourier_data_signal['H1'])

     # noise - 
H1_noise_matrix=np.absolute(fourier_data_noise['H1'])

In [ ]:
L1_signal_matrix=np.absolute(fourier_data_signal['L1'])

     # noise - 
L1_noise_matrix=np.absolute(fourier_data_noise['L1'])

In [ ]:
V1_signal_matrix=np.absolute(fourier_data_signal['V1'])

     # noise - 
V1_noise_matrix=np.absolute(fourier_data_noise['V1'])

In [ ]:
H1_signal_power = 10 * np.log10(H1_signal_matrix)
L1_signal_power = 10 * np.log10(L1_signal_matrix)
V1_signal_power = 10 * np.log10(V1_signal_matrix)

    
    # noise -
    
H1_noise_power = 10 * np.log10(H1_noise_matrix)
L1_noise_power = 10 * np.log10(L1_noise_matrix)
V1_noise_power = 10 * np.log10(V1_noise_matrix)


# H1

In [ ]:
#  H1
plt.figure()
plt.plot(frequency_signal, H1_signal_power, color='black')
plt.xlabel('Signal Frequency (kHz)')
plt.ylabel('Signal power (dB)')
plt.show()

In [ ]:
plt.figure()
plt.plot(frequency_noise, H1_noise_power, color='black')
plt.xlabel('Noise Frequency (kHz)')
plt.ylabel('Noise power (dB)')
plt.show()

# L1

In [ ]:
plt.figure()
plt.plot(frequency_signal, L1_signal_power, color='black')
plt.xlabel('Signal Frequency (kHz)')
plt.ylabel('Signal power (dB)')
plt.show()

In [ ]:
plt.figure()
plt.plot(frequency_noise, L1_noise_power, color='black')
plt.xlabel('Noise Frequency (kHz)')
plt.ylabel('Noise power (dB)')
plt.show()

# V1

In [ ]:
plt.figure()
plt.plot(frequency_signal, V1_signal_power, color='black')
plt.xlabel('Signal Frequency (kHz)')
plt.ylabel('Signal power (dB)')
plt.show()

In [ ]:

plt.figure()
plt.plot(frequency_noise, V1_noise_power, color='black')
plt.xlabel('Noise Frequency (kHz)')
plt.ylabel('Noise power (dB)')
plt.show()